In [5]:
import pybullet as p
import random
import numpy as np
from sim_class import Simulation
import imageio
import sim_class
import time

In [6]:
# class PID:
#     def __init__(self, kp, ki, kd, dt, integral_limit=None, output_limit=None):
#         self.kp = float(kp)
#         self.ki = float(ki)
#         self.kd = float(kd)
#         self.dt = float(dt)

#         self.integral = 0.0
#         self.prev_error = 0.0

#         self.integral_limit = integral_limit  # e.g. 0.2
#         self.output_limit = output_limit      # e.g. 0.05

#     def reset(self):
#         self.integral = 0.0
#         self.prev_error = 0.0

#     def update(self, target, current):
#         error = float(target - current)

#         # Integral
#         self.integral += error * self.dt
#         if self.integral_limit is not None:
#             self.integral = max(-self.integral_limit, min(self.integral, self.integral_limit))

#         # Derivative
#         derivative = (error - self.prev_error) / self.dt
#         self.prev_error = error

#         # PID output
#         output = (self.kp * error) + (self.ki * self.integral) + (self.kd * derivative)

#         # Clamp output (prevents crazy commands)
#         if self.output_limit is not None:
#             output = max(-self.output_limit, min(output, self.output_limit))

#         return output, error


In [7]:
# dt = 0.025
# target = np.array([0.2031, 0.2095, 0.2096], dtype=float)

# pid_x = PID(kp=2.0, ki=0.0, kd=0.5, dt=dt, integral_limit=0.2, output_limit=0.05)
# pid_y = PID(kp=2.0, ki=0.0, kd=0.5, dt=dt, integral_limit=0.2, output_limit=0.05)
# pid_z = PID(kp=2.0, ki=0.0, kd=0.5, dt=dt, integral_limit=0.2, output_limit=0.03)

# tolerance = 0.001  # 1 mm
# max_steps = 4000
# env = Simulation(num_agents=1)
# for step in range(max_steps):
#     start = time.time()
#     pos = np.array(env.get_pipette_position(robotId=env.robotIds[0]))

#     ux, ex = pid_x.update(target[0], pos[0])
#     uy, ey = pid_y.update(target[1], pos[1])
#     uz, ez = pid_z.update(target[2], pos[2])

#     error_vec = np.array([ex, ey, ez])
#     dist = float(np.linalg.norm(error_vec))

#     print(f"[{step:03d}] pos={pos} err={error_vec} |err|={dist:.6f}")

#     if dist < tolerance:
#         print("Target reached within tolerance.")
#         end = time.time()
#         print(f"Time: {end-start:.2f} seconds")
#         break

#     # Your sim mapping: X/Y inverted, Z direct
#     action = np.array([-ux, -uy, uz], dtype=float)

#     state = env.apply_actions(action)
#     time.sleep(dt)


In [8]:
setpoint = np.array([0.2031, 0.2095, 0.2096], dtype=float)

dt = 0.025

# First iteration

In [9]:
Kp = 2
Ki = 0
Kd = 0

tol = 0.0001
env = Simulation(num_agents=1)
try:
    start = time.time()
    # initialize the error, integral, and derivative terms
    prev_error = np.zeros(3)
    integral = np.zeros(3)
    derivative = np.zeros(3)
    while True:
        # read current pipette XYZ position
        pip = np.array(env.get_pipette_position(robotId=env.robotIds[0]))

        # calculate the error
        error = setpoint - pip

        # update the integral term
        integral += error * dt

        # calculate the derivative of the error
        derivative = (error - prev_error)/dt
        prev_error = error

        # calculate the output of the PID controller
        output = np.clip((Kp * error) + (Ki * integral) + (Kd * derivative), -5, 5)
        actions = [[output[0], output[1], output[2], 0]] # x, y, z

        # adjust the pipette position based on the output of the PID controller
        state = env.apply_actions(actions)

        p.stepSimulation()

        # wait for a short time before measuring the temperature again
        time.sleep(dt)
        if np.linalg.norm(error) < tol:
            # np.set_printoptions(suppress=True, formatter={'float': '{:.6f}'.format})
            print(f"Error: x {error[0]:.6f}  y {error[1]:.6f}  z {error[2]:.6f}")
            print("Actuall pipette position:", pip)
            print("Target position:", setpoint)
            end = time.time()
            print(f"Time: {end-start:.2f} seconds")
            break
finally:
    env.close()


Error: x 0.000075  y 0.000004  z 0.000066
Actuall pipette position: [0.20302504 0.20949577 0.20953409]
Target position: [0.2031 0.2095 0.2096]
Time: 27.83 seconds


# Second iteration

In [10]:
Kp = 8
Ki = 0
Kd = 0

tol = 0.0001
env = Simulation(num_agents=1)
try:
    start = time.time()
    # initialize the error, integral, and derivative terms
    prev_error = np.zeros(3)
    integral = np.zeros(3)
    derivative = np.zeros(3)
    while True:
        # read current pipette XYZ position
        pip = np.array(env.get_pipette_position(robotId=env.robotIds[0]))

        # calculate the error
        error = setpoint - pip

        # update the integral term
        integral += error * dt

        # calculate the derivative of the error
        derivative = (error - prev_error)/dt
        prev_error = error

        # calculate the output of the PID controller
        output = np.clip((Kp * error) + (Ki * integral) + (Kd * derivative), -5, 5)
        actions = [[output[0], output[1], output[2], 0]] # x, y, z

        # adjust the pipette position based on the output of the PID controller
        state = env.apply_actions(actions)

        p.stepSimulation()

        # wait for a short time before measuring the temperature again
        time.sleep(dt)
        if np.linalg.norm(error) < tol:
            # np.set_printoptions(suppress=True, formatter={'float': '{:.6f}'.format})
            print(f"Error: x {error[0]:.6f}  y {error[1]:.6f}  z {error[2]:.6f}")
            print("Actuall pipette position:", pip)
            print("Target position:", setpoint)
            end = time.time()
            print(f"Time: {end-start:.2f} seconds")
            break
finally:
    env.close()


Error: x 0.000073  y 0.000048  z 0.000044
Actuall pipette position: [0.20302748 0.20945177 0.20955601]
Target position: [0.2031 0.2095 0.2096]
Time: 7.06 seconds


# Third iteration

In [11]:
Kp = 8
Ki = 0
Kd = 0.5

tol = 0.0001
env = Simulation(num_agents=1)
try:
    start = time.time()
    # initialize the error, integral, and derivative terms
    prev_error = np.zeros(3)
    integral = np.zeros(3)
    derivative = np.zeros(3)
    while True:
        # read current pipette XYZ position
        pip = np.array(env.get_pipette_position(robotId=env.robotIds[0]))

        # calculate the error
        error = setpoint - pip

        # update the integral term
        integral += error * dt

        # calculate the derivative of the error
        derivative = (error - prev_error)/dt
        prev_error = error

        # calculate the output of the PID controller
        output = np.clip((Kp * error) + (Ki * integral) + (Kd * derivative), -5, 5)
        actions = [[output[0], output[1], output[2], 0]] # x, y, z

        # adjust the pipette position based on the output of the PID controller
        state = env.apply_actions(actions)

        p.stepSimulation()

        # wait for a short time before measuring the temperature again
        time.sleep(dt)
        if np.linalg.norm(error) < tol:
            # np.set_printoptions(suppress=True, formatter={'float': '{:.6f}'.format})
            print(f"Error: x {error[0]:.6f}  y {error[1]:.6f}  z {error[2]:.6f}")
            print("Actuall pipette position:", pip)
            print("Target position:", setpoint)
            end = time.time()
            print(f"Time: {end-start:.2f} seconds")
            break
finally:
    env.close()


Error: x 0.000072  y 0.000049  z 0.000044
Actuall pipette position: [0.20302752 0.20945146 0.20955644]
Target position: [0.2031 0.2095 0.2096]
Time: 7.63 seconds


In [12]:
Kp = 8
Ki = 0.5
Kd = 0.5

tol = 0.0001
env = Simulation(num_agents=1)
try:
    start = time.time()
    # initialize the error, integral, and derivative terms
    prev_error = np.zeros(3)
    integral = np.zeros(3)
    derivative = np.zeros(3)
    while True:
        # read current pipette XYZ position
        pip = np.array(env.get_pipette_position(robotId=env.robotIds[0]))

        # calculate the error
        error = setpoint - pip

        # update the integral term
        integral += error * dt

        # calculate the derivative of the error
        derivative = (error - prev_error)/dt
        prev_error = error

        # calculate the output of the PID controller
        output = np.clip((Kp * error) + (Ki * integral) + (Kd * derivative), -5, 5)
        actions = [[output[0], output[1], output[2], 0]] # x, y, z

        # adjust the pipette position based on the output of the PID controller
        state = env.apply_actions(actions)

        p.stepSimulation()

        # wait for a short time before measuring the temperature again
        time.sleep(dt)
        if np.linalg.norm(error) < tol:
            # np.set_printoptions(suppress=True, formatter={'float': '{:.6f}'.format})
            print(f"Error: x {error[0]:.6f}  y {error[1]:.6f}  z {error[2]:.6f}")
            print("Actuall pipette position:", pip)
            print("Target position:", setpoint)
            end = time.time()
            print(f"Time: {end-start:.2f} seconds")
            break
finally:
    env.close()


Error: x -0.000070  y -0.000062  z -0.000036
Actuall pipette position: [0.2031695  0.20956228 0.20963587]
Target position: [0.2031 0.2095 0.2096]
Time: 91.34 seconds
